In [1]:
# Vamos a ver el conocido problema de MNIST en redes neuronales.
# Nosotros le vamos a dar a nuestro modelo un número escrito a mano y
# nuestro algortimo se va a encargar de decirnos cual es.
# Para ello vamos a hacer una red neuronal muy sencilla con tan solo dos
# capas.
# Capa de Entrada: Recibe todos los pixeles de la imagen (28x28 pxl).
# Capa de Salida: Vector de 10 elementos que nos va a decir que número del 0-9 [0,1,2,3,4,5,6,7,8,9] corresponde a la imagen que pusimos.

In [2]:
# Importo la librería de TensorFlow.
# Importo el DataSet de mnist anidado en Tensorflow.examples.tutorials.

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# A nuestra variable "mnist" le vamos a pasar nuestro DataSet.
# Este DataSet contiene todas las imagenes y las etiquetas.
# IMAGEN = <<representación de un 3>>
# ETIQUETA = 3

In [6]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) #La imagen tiene dimensión 28x28.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
# Ahora vamos a empezar a definir nuestra gráfica computacional (planos o elementos que vamos a usar en nuestra red).

In [8]:
x = tf.placeholder(tf.float32, [None,784]) #Imagen del número descompuesta a un vetor.
P = tf.Variable(tf.zeros([784,10])) #Matriz de pesos, 784 para recibir la imagen, 10 por las posibles salidas.
b = tf.Variable(tf.zeros([10])) #Vector con BIAS.
y = tf.matmul(x,P)+b # Operación que se hará en los nodos que reciben entradas.
yR = tf.placeholder(tf.float32, [None,10]) #Matriz con las etiquetas REALES del Dataset.

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# EXPLICACIÓN:
# 1.- Va a hacer una matriz de 784. Como le damos a la red imagenes de 28x28 vamos a convertirla en un gran vector. 28x28=784 elementos. El valor NONE significa que la matriz es variable y va a tomar el número de imágenes que le ingresemos nosotros.
# 2.- Pesos dentro de la matriz que inicio con puros ceros y que va a ser una matriz de 784(pixeles) filas x 10(elementos: 0,1,2,...) columnas.
# 3.- Sesgo de la red. Lo inicializamos todo en ceros y va a ser un vector de 10 elementos.
# 4.- y = Capa de salida. Como sabemos, la fórmula de una neurona es (dato_input X matriz_pesos)+BIAS.
# 5.- yR = Va a ser una matriz NONE(Variable) y con 10 elementos que contendrá nuestras etiquetas del dataset(Para entrenar).

In [10]:
# Ahora vamos a definir las funciones con las cuales
# va a aprender nuestro algoritmo en cada una de las
# iteraciones.
# DEFINIR LA FUNCIÓN DE COSTO ENTROPÍA CRUZADA (CROSS ENTROPY)
# PARA PODER MEDIR EL ERROR. LA SALIDA SERÁ CON SOFTMAX.

In [11]:
softmax = tf.nn.softmax_cross_entropy_with_logits(labels=yR, logits=y)
costo = tf.reduce_mean(softmax)
optimizador = tf.train.GradientDescentOptimizer(0.5).minimize(costo)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
# Softmax nos va a dar el vector de 10 elementos para generar la predicción pero en forma de probabilidades. Nos va a decir la probabilidad de que cada imagen que metamos en input se corresponda con cada uno de los elementos del 0-9.
# Por ejemplo: 7 --> [0.05, 0.03, 0.02, 0.1, 0.07, 0.03, 0.09, (0.61), 0.01, 0.05, 0.04]
# Costo va a ser nuestro función de costo, la cual queremos reducir y que tienda a 0.
# A optimizador le vamos a decir que lo que queremos que optimice sea el coste. SI nuestra gradiente descendiente es 2, nuestra gradiente descendiente se va a encargar de ajustar todos nuestros pesos y valores para reducirlo.

In [13]:
# Ahora vamos a añadir 3 elementos más a la gráfica computacional que van a 
# ayudarnos a generar las predicciones y generar el porcentaje de certeza que
# está teniendo en cada una de nuestras iteraciones de entrenamiento.

In [14]:
prediccion = tf.equal(tf.argmax(y, 1), tf.argmax(yR, 1)) #Nos da el array de booleanos para decirnos cuales están bien y cuales no.
accuracy = tf.reduce_mean(tf.cast(prediccion, tf.float32)) #Nos da el porcentaje sobre el array de predicción.
Produccion = tf.argmax(y, 1)
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [15]:
# Explicación:
# En "produccion", lo que está haciendo argmax es traer el número más alto de todas las probabilidades que nos da Softmax y lo vamos a compar con tf.argmax(yR,1).
# es decir, de mi vector de salida reales contra el vector de mi predicción, quiero que comparemos los dos valores para saber si la predicción se hizo bien o mal.
# Nos va a entregar un array de booleanos.
# "accuracy" la vamos a emplear para medir la certeza. A cast le paso predicción y lo que nos va a decir es si tuvimos por ejemplo 10 aciertos y cinco fallos, tu porcentaje de certeza.
# "Produccion" la vamos a usar para demostrar, una vez tenemos la red neuronal ya hecha, que únicamente pasándole una parámetro nos va a generar esa predicción.

In [16]:
# ENTRENAMIENTO DEL ALGORTIMO:

In [19]:
# Función que usaremos para ver que tan bien va aprendiendo nuestro algoritmo o modelo.
# Cada 50 épocas se va a estar imprimiento el costo (número que queremos reducir), la certeza y el número de época en la que estamos entrenando.
def avance(epoca_i, sess, last_features, last_labels):
    costoActual = sess.run(costo, feed_dict={x: last_features, yR: last_labels})
    Certeza = sess.run(accuracy,feed_dict={x:mnist.validation.images,yR: mnist.validation.labels})
    print('Epoca: {:<4} - Costo: {:<8.3} Certeza: {:<5.3}'.format(epoca_i,costoActual,Certeza))

In [20]:
# La siguiente es la importante ya que aquí es donde empezamos
# a entrenar a nuestro algoritmo.

In [25]:
with tf.Session() as sess:
    sess.run(init)
    for epoca_i in range(1000):
        lotex, lotey = mnist.train.next_batch(100)
        sess.run(optimizador, feed_dict={x: lotex, yR: lotey})
        if (epoca_i%50 == 0):
            avance(epoca_i, sess, lotex, lotey)
    print('RESULTADO FINAL: ',sess.run(accuracy, feed_dict={x: mnist.test.images,yR: mnist.test.labels}))
    print('Resultado de una imagen',sess.run(Produccion,feed_dict={x: mnist.test.images[1].reshape(1,784)}))

Epoca: 0    - Costo: 1.65     Certeza: 0.252
Epoca: 50   - Costo: 0.444    Certeza: 0.873
Epoca: 100  - Costo: 0.23     Certeza: 0.894
Epoca: 150  - Costo: 0.35     Certeza: 0.904
Epoca: 200  - Costo: 0.208    Certeza: 0.907
Epoca: 250  - Costo: 0.353    Certeza: 0.909
Epoca: 300  - Costo: 0.267    Certeza: 0.911
Epoca: 350  - Costo: 0.373    Certeza: 0.914
Epoca: 400  - Costo: 0.287    Certeza: 0.918
Epoca: 450  - Costo: 0.215    Certeza: 0.919
Epoca: 500  - Costo: 0.221    Certeza: 0.917
Epoca: 550  - Costo: 0.332    Certeza: 0.911
Epoca: 600  - Costo: 0.208    Certeza: 0.921
Epoca: 650  - Costo: 0.207    Certeza: 0.921
Epoca: 700  - Costo: 0.315    Certeza: 0.922
Epoca: 750  - Costo: 0.276    Certeza: 0.918
Epoca: 800  - Costo: 0.316    Certeza: 0.918
Epoca: 850  - Costo: 0.323    Certeza: 0.919
Epoca: 900  - Costo: 0.305    Certeza: 0.92 
Epoca: 950  - Costo: 0.275    Certeza: 0.92 
RESULTADO FINAL:  0.9146
Resultado de una imagen [2]


In [26]:
# Explicación:
#
# Inicio las variables con nuestro iniciador.
# Vamos a entrenar 1000 veces nuestros datos de entrenamiento.
# En el ciclo vamos a tener el loteX y el loteY; para cada recorrido que 
# vamos a tener en nuestra red neuronal le vamos a pasar solo estos dos lotes.
# No le vamos a pasar el DataSet entero porque requiere mucha potencia de cálculo.
# Le vamos a ir pasando de 100 en 100 imagenes (porque no tengo los servidores de Google ;-) ).
# Ahora vamos a decirle, a través de la Session() que queremos correr el optimizador y lo
# vamos a alimentar ("x: " como entrada) con el loteX y a las etiquetas ("yR: ") con el loteY.
# Optimizador depende de costo. Costo depende de Softmax. Softmax depende de yR. yR depende de y. y depende de (x,P)+b. 
# Así va a ir iterando en cada una de nuestras épocas.
# Le decimos que si nuestra época es el módulo de 50 llame a la función "avance".(Para que cada 50 épocas imprima el costo y certeza).
# Por último imprimo nuestro resultado final, sobre nuestro DataSet de testing, que tan bien ha aprendido nuestra red neuronal.
# También imprimo el resultado de esa imagen. Solo le mandamos una imágen.

In [27]:
# Epoca 0: De 100 imágenes que se le mandaron, solo se pudieron acertar el 46%. Bajo el costo (1.71).
# Epoca 50: De 100 imágenes que se le mandaron, solo se puedieron acercar el 88%. Bajo el costo (0.487).
# ...
# Nos dice que de nuestro set de datos solo se pudo predecir bien el 91.5%.
# Al hacer ".reshape" le voy a decir que solo le voy a enviar un renglón (una imágen) con 784 columnas (28x28).
# Vamos  a ver las etiquetas para comprobar que lo ha hecho bien:

In [28]:
mnist.test.labels[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
# Correcto. Es el 2.